<a href="https://colab.research.google.com/github/titsitits/Python_Data_Science/blob/master/8_Example_Ames_Housing_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mickaël Tits
CETIC
mickael.tits@cetic.be

# Ames (Iowa) Housing Dataset

Plus d'informations ici:
http://jse.amstat.org/v19n3/decock.pdf

Détails sur les variables: https://github.com/titsitits/Python_Data_Science/blob/master/Donn%C3%A9es/data_description.txt

## Préparation/exploration du dataset

In [0]:
import pandas as pd

ames = pd.read_csv("https://raw.githubusercontent.com/titsitits/Python_Data_Science/master/Donn%C3%A9es/train.csv")

In [0]:
ames.count().values

In [0]:
#another visualization library
import seaborn as sns

sns.set(rc={'figure.figsize':(5,5)})
sns.distplot(ames.SalePrice, bins=30)

In [0]:
from sklearn.model_selection import train_test_split

train_ames, val_ames = train_test_split(ames, test_size = 0.2, random_state=5)

In [0]:
sns.distplot(train_ames.SalePrice, bins=30)
sns.distplot(val_ames.SalePrice, bins=30)


## Exploration de base: analyse de corrélation (variables continues)

Remarque: la méthode pandas.DataFrame.corr() omet automatiquement les variables non-numériques.

In [0]:

correlation_matrix = train_ames.corr()

#corrélations des variables avec le prix
corr_with_price = correlation_matrix["SalePrice"]

#On trie les variables selon la valeur absolue de leur corrélation avec le prix
best_features = corr_with_price.abs().sort_values(ascending=False)

# On ne évidemment peut utiliser le label comme variable prédictive
best_features = best_features.drop("SalePrice")

plt.figure(figsize=(8,8))
best_features.plot.bar()


In [0]:
fig,axes = plt.subplots(1,2, figsize=(10,5))
ames.plot.scatter("OverallQual","SalePrice", ax = axes[0])
ames.plot.scatter("GrLivArea","SalePrice", ax = axes[1])



Note: On remarque que deux points pourraient être considérés comme des outliers: ils s'écartent fortement de la relation linéaire globale entre les deux variables. Une question qui se pose naturellement: A quels points du premier graphe correspondent ceux du deuxième graphe (en particulier les outliers) ?

La librairie moderne de visualition graphique **`atlair`** permet de répondre à ce genre de question.


### Bonus :  visualisation moderne (interactive) avec `atlair`

In [0]:
# load an example dataset

import altair as alt

interval = alt.selection_interval()

chart = alt.Chart(ames).mark_point().encode(  y='SalePrice',
  color=alt.condition(interval, 'SaleCondition', alt.value('lightgray')) ).properties( selection=interval)

chart.encode(x='GrLivArea') | chart.encode(x='OverallQual')

#Les graphiques ci-dessous

#N'hésitez pas à tester une autre catégorie (au lieu de MSZoning) pour définir la légende (par exemple: SaleCondition )

Les graphiques ci-dessus sont interactifs. Vous pouvez sélectionner une partie des points avec votre souris. 
On remarque que les deux outliers en terme de ground_living_area sont le même qu'en terme de overall_quality. 

In [0]:
is_outlier = (ames["GrLivArea"] > 4000) & (ames["SalePrice"] < 300000)
display(ames[is_outlier == True])

In [0]:
#mode: valeur la plus fréquente
ames.mode().iloc[0:1]

Une comparaison rapide entre les outliers et les catégories les plus fréquentes montrent notamment: les outliers sont construits sur des terrains irréguliers (LotShape = IR3, et LandContour = Lvl), et les bâtiments ne sont pas finis (SaleType = New, SaleCondition = Partial). Ces particularités peuvent expliquer leur prix.

## Préparation des sets d'entraînement et de validation

In [0]:
#garder les outliers ou pas ?

ames2 = ames[~is_outlier]

#ames2 = ames

### Sélection des variables prédictives (features)

In [0]:
#Prenons les cinq meilleures variables prédictives (en sa basant sur leur corrélation avec le prix)
featurelist = best_features.index[:5].to_list()

dataset = ames2[featurelist+["SalePrice"]]
dataset = dataset.dropna()

X = dataset[featurelist]
y = dataset["SalePrice"]

dataset.describe()

### Séparation des données d'entraînement et de validation

In [0]:
from sklearn.model_selection import train_test_split

Xtrain, Xval, ytrain, yval = train_test_split(X, y, test_size = 0.2, random_state=5)

## Régression linéaire

In [0]:
from sklearn.linear_model import LinearRegression

price_predictor = LinearRegression()
price_predictor.fit(Xtrain, ytrain)


In [0]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error


print("R2 Scores (train, val):", price_predictor.score(Xtrain, ytrain), price_predictor.score(Xval, yval))


y_pred = price_predictor.predict(Xval)

print("biais:", np.mean(y_pred - yval.values) ) #si il est négatif: sous-évaluation (en moyenne), si il est positif: sur-évaluation
print("MAE:", mean_absolute_error(yval.values, y_pred) )
print("RMSE:", np.sqrt(mean_squared_error(yval.values, y_pred)) )

#Tester en gardant ou en retirant les outliers; tester avec différentes ensembles de variables

## Comparaison pour différents nombres de variables prédictives

In [0]:
perfs = pd.DataFrame(columns = ['train','val'])

for n in range(1,len(best_features)):

  featurelist = best_features.index[:n].to_list()
  
  dataset = ames2[featurelist+["SalePrice"]]
  dataset = dataset.dropna()
  
  X = dataset[featurelist]
  y = dataset["SalePrice"]
  
  Xtrain, Xval, ytrain, yval = train_test_split(X, y, test_size = 0.2, random_state=5)
  price_predictor.fit(Xtrain, ytrain)
  perfs.loc[n] = price_predictor.score(Xtrain, ytrain), price_predictor.score(Xval, yval)

perfs.plot()

On remarque que les performances en validation grimpent fortement lors de l'ajout de la 11ème variable, et tombent lors de l'ajout de la 15ème. Vérifions les performances en omettant la 15ème variable.

In [0]:
print(best_features.index[14])

In [0]:
featurelist = best_features.index.to_list()

featurelist.remove('LotFrontage')
featurelist.remove('Id')

dataset = ames2[featurelist+["SalePrice"]]
dataset = dataset.dropna()

X = dataset[featurelist]
y = dataset["SalePrice"]

Xtrain, Xval, ytrain, yval = train_test_split(X, y, test_size = 0.2, random_state=5)
price_predictor.fit(Xtrain, ytrain)

price_predictor.score(Xtrain, ytrain), price_predictor.score(Xval, yval)

In [0]:
plt.scatter(yval,price_predictor.predict(Xval))

On constate que le nuage de doit semble former une légère courbe. Cela semble indiquer qu'il existe des relations non-linéaires, telles que des relations polynomiales entre les variables et le prix. La régression linéaire se base en effet sur l'hypothèse très simpliste que les relations seraient linéaires. Si on revisualise les nuages de points plus haut montrant l'interactions entre différentes variables (comme 'OverallQual') et le prix, on constate que la relation et plutôt quadratique que linéaire.

Une variation de la régression linéaire, appelée régression polynomiale, permet de facilement prendre en compte de potentielles interactions non-linéaires entre les variables, en créant de nouvelles variables à partir de polynômes des variables d'origine. Bien sûr, les possibilités ne se liminent pas aux polynômes.

##Régression polynomiale

In [0]:


X = dataset[featurelist]

def add_polynomials(X):
  for col in X.columns:
    #prise en compte des carrés des variables
    X[col+"_square"] = X[col]**2
    #prise en compte des racines carrées
    X[col+"_sqrt"] = X[col]**(1/2)
    
  return X

X = add_polynomials(X)

len(X.columns)

In [0]:
Xtrain, Xval, ytrain, yval = train_test_split(X, y, test_size = 0.2, random_state=5)
price_predictor.fit(Xtrain, ytrain)

print("performances (train,val):", price_predictor.score(Xtrain, ytrain), price_predictor.score(Xval, yval))
plt.scatter(yval,price_predictor.predict(Xval))

## Comparaison pour différents nombres d'exemples (taille du training set)

In [0]:
perfs2 = pd.DataFrame(columns = ['train','val'])

for nsamples in range(10,800,10):
  
  trainset, valset = train_test_split(dataset, test_size = 550, random_state=5)
  
  #On extrait un sample sur le training set
  trainset_sample = trainset.sample(nsamples)
  
  Xtrain = trainset_sample[featurelist]
  ytrain = trainset_sample["SalePrice"]
  
  Xval = valset[featurelist]
  yval = valset["SalePrice"]
  
  price_predictor.fit(Xtrain, ytrain)
  perfs2.loc[nsamples] = price_predictor.score(Xtrain, ytrain), price_predictor.score(Xval, yval)
  
perfs2.plot()

On constate que les performances diminuent avec le nombre d'échantillons d'entraînement, et que les performances en validation sont au départ beaucoup plus faibles, ce qui est signe d'**overfitting**. En effet, en utilisant seulement 50 exemples (gauche du graphe), on remarque que le modèle compte presque autant de paramètres (un par variable prédictive, soit 36). Dans ce cas, les paramètres peuvent être adaptés aux exemples spécifiques d'entraînement pour apprendre leur prédiction presque "par coeur". Le modèle n'est alors pas générique et fonctionne mal sur de nouvelles données (comme le montrent les faibles performances en validation). On constate ensuite que pour 400 échantillons d'entraînement, les performances en entraînement se stabilisent, et les performances en validation sont équivalentes, ce qui signifie que le nombre d'échantillons est suffisant (plus d'overfitting).

In [0]:
len(price_predictor.coef_)

## Performances réelles

Pour évaluer les performances réelles du modèles, nous pouvons utiliser un set de données qui n'a pas encore été utilisé, ni pour l'entraînement ni pour la validation. Cela permet de simuler une utilisation réelle du modèle, sur de nouvelles maisons (dont on voudrait estimer le prix). L'évaluation des performances à partir du set de validation seraient biaisée, car il a en effet permis de choisir les paramètres du modèles, en l'occurence le choix des variables prédictives (le modèle a donc été indirectement optimisé pour ces données spécifiques).

Sur la plateforme [Kaggle](https://www.kaggle.com/), pour garantir l'absence de biais lors du design du modèle, un set de test est généralement fourni séparément, et les labels ne sont volontairement pas fournis. L'utilisateur de la plateforme doit alors soumettre à la plateforme les prédictions données par le modèle développé, et reçoit le résultat de l'évaluation effectuée par la plateforme. Cela permet notamment de classer différents compétiteurs lors d'un concours: https://www.kaggle.com/competitions

Concernant le dataset utilisé, on peut soumettre des prédictions ici: https://www.kaggle.com/c/home-data-for-ml-course/overview/evaluation




In [0]:
#Métrique utilisée sur Kaggle: 
#Submissions are evaluated on Root-Mean-Squared-Error (RMSE) between the logarithm of the predicted value and the logarithm of the observed sales price. 
#(Taking logs means that errors in predicting expensive houses and cheap houses will affect the result equally.)

def metric(a,b):
  
  a = np.log(a)
  b = np.log(b)
  return np.sqrt(mean_squared_error(a,b))

#Mean absolute error (plus intuitif)
def MAE(a,b):
  return mean_absolute_error(a,b)


In [0]:
# Vérifions déjà les performances sur le set de validation

featurelist = best_features.index[:n].to_list()

dataset = ames[featurelist+["SalePrice"]]
dataset = dataset.dropna()

X = dataset[featurelist]

y = dataset["SalePrice"]

Xtrain, Xval, ytrain, yval = train_test_split(X, y, test_size = 0.4)

price_predictor.fit(Xtrain, ytrain)

trainpreds = price_predictor.predict(Xtrain)
valpreds = price_predictor.predict(Xval)

#Pour éviter de prédire des valeurs anormales, on limite les prédictions au range du set d'entraînement
trainpreds = trainpreds.clip(ytrain.min(), ytrain.max())
valpreds = valpreds.clip(ytrain.min(), ytrain.max())


MAE(trainpreds,ytrain.values), MAE(valpreds,yval.values)

In [0]:
#uncomment to use polynomials
X = dataset[featurelist]
X = add_polynomials(X)

Xtrain, Xval, ytrain, yval = train_test_split(X, y, test_size = 0.4)

poly_predictor = LinearRegression()
poly_predictor.fit(Xtrain, ytrain)

trainpreds = poly_predictor.predict(Xtrain)
valpreds = poly_predictor.predict(Xval)

#Pour éviter de prédire des valeurs anormales, on limite les prédictions au range du set d'entraînement
trainpreds = trainpreds.clip(ytrain.min(), ytrain.max())
valpreds = valpreds.clip(ytrain.min(), ytrain.max())


metric(trainpreds,ytrain.values), metric(valpreds,yval.values)

Remarquez que vous obtiendrez des résultats différents à chaque fois que vous relancez les cellules ci-dessus: cela est dû à la séparation aléatoire des sets d'entraînement et de validation. 

Afin d'avoir un indicateur plus robuste du modèle, une possibilité serait de calculer une moyenne de ces résultats, ou d'utiliser un processus **leave-one-out** (voir Chapitre précédent).

In [0]:
testset = pd.read_csv("https://raw.githubusercontent.com/titsitits/Python_Data_Science/master/Donn%C3%A9es/test.csv")

testset = testset[featurelist].fillna(0)
testpreds = price_predictor.predict(testset)

testpreds = testpreds.clip(ytrain.min(), ytrain.max())

submission = testset
submission["SalePrice"] = testpreds
submission = submission[["Id","SalePrice"]]
submission.to_csv("mysubmission_linear_regression.csv", index = False)

In [0]:
testset = pd.read_csv("https://raw.githubusercontent.com/titsitits/Python_Data_Science/master/Donn%C3%A9es/test.csv")

testset = testset[featurelist].fillna(0)

testset2 = add_polynomials(testset)
testpreds = poly_predictor.predict(testset2)

testpreds = testpreds.clip(ytrain.min(), ytrain.max())

submission = testset
submission["SalePrice"] = testpreds
submission = submission[["Id","SalePrice"]]
submission.to_csv("mysubmission_polynomial_regression.csv", index = False)

## Exploration de base des variables catégorielles

In [0]:
#Analysons l'effet de chaque variable catégorielle

from matplotlib import pyplot as plt



all_columns = ames.columns
cont_columns = featurelist

cat_columns = set(all_columns) - set(cont_columns) - set(['LotFrontage'])
cat_columns = list(cat_columns)
cat_columns.remove("SalePrice")

diffs_per_group = []

for cat in cat_columns:  
  
  group_means = ames.groupby(cat)["SalePrice"].mean()
  diffs_per_group.append(group_means.max() - group_means.min())
  
  #Bonus: vous pouvez afficher les graphes pour chaque groupe
  #plt.figure(figsize=(5,5))
  #group_means.plot.bar()


In [0]:
diffs_per_group = np.array(diffs_per_group)
best_cats = diffs_per_group.argsort()[::-1]
best_cats_list = [cat_columns[i] for i in best_cats]
print(best_cats_list)

In [0]:
#ames["PoolQC"]

In [0]:
ames["PoolQC"] = ames["PoolQC"].fillna("NP") #no pool

In [0]:
ames.groupby("PoolQC")["SalePrice"].mean()

## Bonus - Un algorithme plus moderne: catboost

In [0]:
#Installer un nouveau package: on appelle une ligne de commande linux grâce au symbole "!". On utilise le gestionnaire de packages python pip pour installer un nouveau package
!pip install catboost

In [0]:
contfeatures = featurelist#[:10]
catfeatures = best_cats_list#[:10]

newfeaturelist =  contfeatures + catfeatures

In [0]:
#garder les colonnes qui contiennent au moins 90% de données valides

tmp = ames[newfeaturelist]
good_cols = tmp.count() > 0.9*len(tmp)

goodfeatures = good_cols.index[good_cols == True].to_list()

newfeaturelist = list(set(goodfeatures)) #on s'assure de ne pas avoir de doublons en convertissant temporairement en set

In [0]:
dataset = ames[newfeaturelist+["SalePrice"]]
dataset = dataset.fillna(0)

X = dataset[newfeaturelist]

y = dataset["SalePrice"]

Xtrain, Xval, ytrain, yval = train_test_split(X, y, test_size = 0.4, random_state=5)

In [0]:
import numpy as np
from catboost import Pool, CatBoostRegressor


cat_feature_ids = [i for i in range(len(newfeaturelist)) if newfeaturelist[i] in catfeatures]


train_pool = Pool(Xtrain.values, ytrain.values, cat_features=cat_feature_ids)
val_pool = Pool(Xval, yval, cat_features=cat_feature_ids) 
all_pool = Pool(X, y, cat_features=cat_feature_ids) 

# specify the training parameters 
model = CatBoostRegressor(iterations=300, 
                          depth=3, 
                          learning_rate=0.2, 
                          loss_function='RMSE')
#train the model
model.fit(train_pool, silent=True)

In [0]:
# make the prediction using the resulting model
trainpreds = model.predict(train_pool)
valpreds = model.predict(val_pool)

rmselog(trainpreds,ytrain.values), rmselog(valpreds,yval.values)

In [0]:
# specify the training parameters 
model = CatBoostRegressor(iterations=300, 
                          depth=3, 
                          learning_rate=0.2, 
                          loss_function='RMSE')

#entraînement du modèle, avec un critère d'arrêt lorsque les performances en validation baissent: éviter le surentraînement (overfitting)
model.fit(train_pool,eval_set = val_pool, early_stopping_rounds = 100, silent = True)

In [0]:
# make the prediction using the resulting model
trainpreds = model.predict(train_pool)
valpreds = model.predict(val_pool)

#Pour éviter de prédire des valeurs anormales, on limite les prédictions au range du set d'entraînement
trainpreds = trainpreds.clip(ytrain.min(), ytrain.max())
valpreds = valpreds.clip(ytrain.min(), ytrain.max())

rmselog(trainpreds,ytrain.values), rmselog(valpreds,yval.values)

### Soumission des résultats

In [0]:
testset = pd.read_csv("https://raw.githubusercontent.com/titsitits/Python_Data_Science/master/Donn%C3%A9es/test.csv")

Xtest = testset[newfeaturelist].fillna(0)
test_pool = Pool(Xtest.values, cat_features=cat_feature_ids)

#Pour la soumission, on peut éventuellement réentraîner le modèle sur toutes les données d'entraînement et de validation (pour espérer avoir un modèle plus générique. Cependant: attention à l'overfitting sans critère d'arrêt)
model.fit(all_pool, silent = True)

#Prédictions
testpreds = model.predict(test_pool)

submission = testset
submission["SalePrice"] = testpreds
submission = submission[["Id","SalePrice"]]
submission.to_csv("mysubmission_catboost.csv", index = False)

Les résultats avec le fichier de baseline (sample_submission.csv) ont été obtenus avec une régression linéaire sur l'année et le mosi de vente, la surface du lot et le nombre de chambres. Les résultats de la soumission sur Kaggle donnent: 

`rmselog = 0.40890`

La soumission avec catboost sur ['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', 'MSZoning', 'PoolQC', 'SaleType', 'SaleCondition', 'LotShape'] donne

`rmselog = 0.25360`

L'algorithme proposé prédit donc mieux les maisons que la baseline.

Bien évidemment, ce n'est qu'un essai, et il existe de très nombreuses possibilités pour améliorer le modèle:

* Vérifier les données et supprimer d'éventuelles anomalies
* Tester d'autre ensembles de variables (en explorant plus profondément leurs relations et leur influence sur le prix)
* Créer de nouvelles variables pertinentes. Par exemple, à partir de la variable catégorielle "neighboorhood" et du prix moyen de chaque quartier, on pourrait extraire une variable numérique indiquant le "standing" du quartier. On pourrait aussi traduire les nombreuses variables indiquant une échelle de qualité ("Excellent","Typical","Fair", ...) en variables numériques. On pourrait également extraire des variables "dummy" sur d'autres catégories.
* Les variables sont peut-être redondantes (elles apportent la même information), ce qui complexifie inutilement le modèle. Il peut être intéressant d'extraire un ensemble de variables apportant un maximum d'informations non-redondantes.
* Tester d'autres algorithmes de machine learning




## Bonus - Exploration de la redondance

L'analyse de corrélation permet de sélectionner facilement des variables prédictives pertinentes, comme nous l'avons vu plus haut.
Une limitation importante de cette méthode de sélection de variables est la non-prise en compte de la redondance possible entre les variables prédictives. Deux variables redondantes signifie qu'elles contiennent en partie la même information (elles sont donc généralement fortement corrélées entre elles). La sélection d'une variable redondante apporte donc peu d'information supplémentaire pour l'entraînement du modèle prédictif. au contraire, l'augmentation du nombre de variables rend alors le modèle inutilement plus complexe, ce qui rend l'entraînement plus lourd, et augmente les risques d'overfitting.

Dans les exemples proposés ci-dessous, nous avons notamment utilisé deux variables particulièrement redondantes: 'GarageCars' et 'GarageArea', qui apportent presque la même information (la surface qu garage vs le nombre de voitures qu'on peut y mettre). On peut d'ailleurs remarquer une très forte corrélation entre ces deux variables:

In [0]:
ames[["GarageCars","GarageArea"]].corr()

Lors de l'analyse de corrélation, il est donc également intéressant de vérifier que les variables sélectionnées ne soient donc pas redondantes. On peut notamment s'attendre à ce que les années de construction de la maison et des garages soient redondantes, tout comme différentes variables liées à la surface de la maison ou au nombre de pièces.

In [0]:
import seaborn as sns

def plot_corr(df, absolute = False, size = (15,15), decimal = 1):
  
  correlation_matrix = df.corr().round(decimal)
  if absolute:
    correlation_matrix = correlation_matrix.abs()
  plt.figure(figsize = size)
  sns.heatmap(correlation_matrix, annot=True)
             
plot_corr(ames, absolute = True)

### Bonus - L'analyse en composantes principales (PCA) 

Une manière de se débarrasser du problème est l'utilisation de l'analyse en composantes principales (PCA, aussi appelée Singular value Decomposition - SVD). C'est un algorithme d'algèbre matriciel (souvent considéré comme une technique de machine learning non-supervisé), permettant d'extraire un nouvel ensemble de variables **orthogonales** (i.e. non-corrélées entre elles), à partir de combinaisons linéaires des variables d'origine. Dans ce nouvel ensemble de variables, on peut alors sélectionner les variables prédictives sans le moindre risque de redondance.

In [0]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

dataset = ames[featurelist+["SalePrice"]]
dataset = dataset.dropna()

X = dataset[featurelist]
y = dataset["SalePrice"]

scaler = StandardScaler()

X = scaler.fit_transform(X)

Xtrain, Xval, ytrain, yval = train_test_split(X, y, test_size = 0.2, random_state=5)

#On réalise la PCA sur les données d'entraînement
pca = PCA(n_components=X.shape[1]-2)
pca.fit(Xtrain)
Xtrainpcs = pca.transform(Xtrain)
Xtrainpcs = pd.DataFrame(Xtrainpcs)

In [0]:
plot_corr(Xtrainpcs, absolute = True, size = (10,10))

In [0]:
Xtrainpcs["SalePrice"] = ytrain

correlation_matrix = Xtrainpcs.corr()
corr_with_price = correlation_matrix["SalePrice"].abs()
best_pcs = corr_with_price.sort_values(ascending=False)

# On ne évidemment peut utiliser la label comme variable prédictive
best_pcs = best_pcs.drop("SalePrice")

best_pcs.plot.bar()

In [0]:
pcslist = best_pcs.index[:25].to_list()

Xpcs = pca.transform(X)
Xpcs = pd.DataFrame(Xpcs)
Xfeatures = Xpcs[pcslist]

Xtrain, Xval, ytrain, yval = train_test_split(Xfeatures, y, test_size = 0.3)

#Entraînement
price_predictor.fit(Xtrain, ytrain)

#Prédictions
trainpreds = price_predictor.predict(Xtrain)
valpreds = price_predictor.predict(Xval)

#Pour éviter de prédire des valeurs anormales, on limite les prédictions au range du set d'entraînement
trainpreds = trainpreds.clip(ytrain.min(), ytrain.max())
valpreds = valpreds.clip(ytrain.min(), ytrain.max())

rmselog(trainpreds, ytrain.values) , rmselog(valpreds, yval.values)

### Bonus - PLSRegression

In [0]:
from sklearn.cross_decomposition import PLSRegression

pls_predictor = PLSRegression(max_iter=200, n_components=20)

dataset = ames[featurelist+["SalePrice"]]
dataset = dataset.dropna()

X = dataset[featurelist]

scaler = StandardScaler()
X = scaler.fit_transform(X)

y = dataset["SalePrice"]

Xtrain, Xval, ytrain, yval = train_test_split(X, y, test_size = 0.4)
pls_predictor.fit(Xtrain, ytrain)

#Prédictions
trainpreds = pls_predictor.predict(Xtrain)
valpreds = pls_predictor.predict(Xval)

#Pour éviter de prédire des valeurs anormales, on limite les prédictions au range du set d'entraînement
trainpreds = trainpreds.clip(ytrain.min(), ytrain.max())
valpreds = valpreds.clip(ytrain.min(), ytrain.max())

rmselog(trainpreds, ytrain.values) , rmselog(valpreds, yval.values)

# Poubelle

In [0]:

#optional: add polynomials (only to continuous variables)
poly = False
if poly:
  goodcontfeatures = [g for g in goodfeatures if g in contfeatures]
  Xcont = dataset[goodcontfeatures]
  Xcont = add_polynomials(Xcont)
  goodcatfeatures = [g for g in goodfeatures if g in catfeatures]
  Xcat = dataset[goodcatfeatures]
  X = pd.concat([Xcont, Xcat], axis=1)
  newfeaturelist = X.columns